In [1]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components=2)

In [2]:
from VAE.Model import ModelDesc, Trainer  # , RandomZData
# from VAE.visualize_params import get_default_hparams
from VAE.extra_info_params import get_default_hparams

hps = get_default_hparams()

OpenCV is built with OpenMP support. This usually results in poor performance. For details, see https://github.com/tensorpack/benchmarks/blob/master/ImageNet/benchmark-opencv-resize.py


In [3]:
checkpoint_path = './train_log/train_5m/checkpoint'

In [4]:
from VAE.load_data import *
def get_data(hps):
    dfX, df_next_deltaClose = load_data_seq(hps)

    segment, next_segment, target_one_hot = segment_seq(dfX, df_next_deltaClose, hps)

    train_segment, test_segment, _, _, train_target_one_hot, test_target_one_hot =\
    train_test_split(segment, next_segment, target_one_hot, hps)

    return train_segment, train_target_one_hot, test_segment, test_target_one_hot

In [5]:
X_train, y_train, X_val, y_val = get_data(hps)

Normalize: Z score


In [6]:
from tensorpack.predict.config import PredictConfig
from tensorpack.tfutils import SmartInit
from tensorpack.predict import OfflinePredictor

In [7]:
M = ModelDesc(hps)

straight_forward_z = PredictConfig(
        session_init=SmartInit(checkpoint_path),
        model=M,
        input_names=['x'],
        output_names=['encode/add_1']
)

z_cal = OfflinePredictor(straight_forward_z)

[1223 00:26:32 @collection.py:146] New collections created in tower : tf.GraphKeys.MODEL_VARIABLES of size 10, tf.GraphKeys.METRIC_VARIABLES of size 2
[1223 00:26:32 @collection.py:165] These collections were modified but restored in : (tf.GraphKeys.SUMMARIES: 0->7)
[1223 00:26:32 @sessinit.py:87] WRN The following variables are in the checkpoint, but not found in the graph: global_step, optimize/beta1_power, optimize/beta2_power
[1223 00:26:32 @sessinit.py:114] Restoring checkpoint from ./train_log/train_5m/model-50000 ...
INFO:tensorflow:Restoring parameters from ./train_log/train_5m/model-50000


In [ ]:
Z = z_cal(X_train)[0]
Z.shape

(184399, 120)

In [ ]:
np.savez('Z_5m.npz', Z = Z, y = y_train)

In [ ]:
pca = PCA(n_components=2)
pca.fit(Z)
z_2d = pca.transform(Z) 

In [ ]:
import pandas as pd
df = pd.DataFrame()
df['z_0'] = z_2d[:,0]
df['z_1'] = z_2d[:,1]
df['y']   = np.argmax(y_train, axis= -1)

In [ ]:
# %matplotlib inline
import seaborn as sns
ax = sns.scatterplot(x="z_0", y="z_1", hue="y", data=df)
ax.savefig("z_2d_scatter.png")